# OpenAI Agent SDK with Guardrails - Proof of Concept

This notebook demonstrates a well-designed, production-ready implementation of agents with swappable LLM providers and guardrails.

## Architecture Overview

The implementation follows SOLID principles with swappable components:

1. **LLM Providers**: Abstract interface allowing OpenAI, Ollama, or other providers
2. **Guardrail Providers**: Pluggable guardrails using OpenAI, Gemini, or local models
3. **Agent System**: Agents loaded from SuperPrompt templates with tool support
4. **Tool System**: File operations and inter-agent communication

## Key Features

- ✅ Swappable LLM providers (OpenAI, Ollama)
- ✅ Swappable guardrail providers (OpenAI, Gemini, Local)
- ✅ Agent creation from SuperPrompt files
- ✅ Agent-to-agent communication and delegation
- ✅ File-saving tools
- ✅ Prompt composition and reuse
- ✅ Input/Output guardrails

## Implementation

The core implementation is in `my_chat_gpt_utils/agents_sdk.py` for better modularity and reusability.

## 1. Setup and Imports

In [ ]:
import sys
import os
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, os.path.abspath('..'))

# Import our agents SDK
from my_chat_gpt_utils.agents_sdk import (
    # Providers
    OpenAIProvider,
    OllamaProvider,
    OpenAIGuardrailProvider,
    GeminiGuardrailProvider,
    LocalGuardrailProvider,
    # Core components
    Agent,
    AgentConfig,
    AgentOrchestrator,
    SuperPromptLoader,
    ToolRegistry,
    # Data models
    Message,
    GuardrailResult,
)

print("✓ Imports successful")

## 2. Initialize LLM Providers

The design allows you to easily swap between different LLM providers.

In [ ]:
# Option 1: OpenAI Provider
try:
    openai_provider = OpenAIProvider(model="gpt-4o-mini")
    print("✓ OpenAI provider initialized")
except Exception as e:
    print(f"⚠ OpenAI provider failed: {e}")
    openai_provider = None

# Option 2: Ollama Provider (requires Ollama running locally)
try:
    ollama_provider = OllamaProvider(model="llama3.2")
    print("✓ Ollama provider initialized")
except Exception as e:
    print(f"⚠ Ollama provider failed: {e}")
    ollama_provider = None

# Select which provider to use
llm_provider = openai_provider if openai_provider else ollama_provider

if llm_provider:
    print(f"\n✓ Using LLM provider: {llm_provider.__class__.__name__}")
else:
    print("\n⚠ No LLM provider available. Please configure OpenAI or Ollama.")

## 3. Initialize Guardrail Providers

Guardrails can be swapped to use different providers (e.g., use Gemini for guardrails on OpenAI prompts).

In [ ]:
# Option 1: OpenAI Guardrails (moderation API)
try:
    openai_guardrail = OpenAIGuardrailProvider()
    print("✓ OpenAI guardrail provider initialized")
except Exception as e:
    print(f"⚠ OpenAI guardrail failed: {e}")
    openai_guardrail = None

# Option 2: Gemini Guardrails
try:
    gemini_guardrail = GeminiGuardrailProvider()
    print("✓ Gemini guardrail provider initialized")
except Exception as e:
    print(f"⚠ Gemini guardrail failed: {e}")
    gemini_guardrail = None

# Option 3: Local Guardrails (always available)
local_guardrail = LocalGuardrailProvider()
print("✓ Local guardrail provider initialized")

# Select guardrails (you can mix and match)
input_guardrail = openai_guardrail if openai_guardrail else local_guardrail
output_guardrail = local_guardrail  # Use local for output by default

print(f"\n✓ Using input guardrail: {input_guardrail.__class__.__name__}")
print(f"✓ Using output guardrail: {output_guardrail.__class__.__name__}")

## 4. Initialize Core Components

In [ ]:
# Initialize components
prompt_loader = SuperPromptLoader()
tool_registry = ToolRegistry()
orchestrator = AgentOrchestrator()

print("✓ Core components initialized\n")
print("Available SuperPrompts:")
prompts = prompt_loader.list_prompts()
for i, prompt in enumerate(prompts[:10], 1):  # Show first 10
    print(f"  {i}. {prompt}")
if len(prompts) > 10:
    print(f"  ... and {len(prompts) - 10} more")

## 5. Example 1: Create Agent from SuperPrompt

Agents are created from SuperPrompt templates, with tools and guardrails.

In [ ]:
# Create an architect agent from SuperPrompt
architect_config = AgentConfig(
    name="Software Architect",
    system_prompt=prompt_loader.load_prompt("software_architect_ai_prompt.txt"),
    temperature=0.7,
    tools=tool_registry.get_tool_definitions()
)

architect_agent = orchestrator.create_agent(
    name="architect",
    config=architect_config,
    llm_provider=llm_provider,
    tool_registry=tool_registry,
    input_guardrail=input_guardrail,
    output_guardrail=output_guardrail
)

print("✓ Architect agent created with:")
print(f"  - LLM: {llm_provider.__class__.__name__ if llm_provider else 'None'}")
print(f"  - Input Guardrail: {input_guardrail.__class__.__name__}")
print(f"  - Output Guardrail: {output_guardrail.__class__.__name__}")
print(f"  - Tools: {len(tool_registry.get_tool_definitions())}")

In [ ]:
# Test the architect agent
if llm_provider:
    response = architect_agent.run(
        "Design a high-level architecture for a simple REST API with user authentication. "
        "Save the architecture to a file called 'api_architecture.md'."
    )
    print("\nArchitect Response:")
    print(response)
else:
    print("⚠ No LLM provider available. Please configure OpenAI or Ollama.")

## 6. Example 2: Multi-Agent Collaboration

Multiple agents can work together, each with their own specialization.

In [ ]:
# Create a coder agent
coder_config = AgentConfig(
    name="Python Programmer",
    system_prompt=prompt_loader.load_prompt("python-programmer-superprompt.md"),
    temperature=0.3,  # Lower temperature for code generation
    tools=tool_registry.get_tool_definitions()
)

coder_agent = orchestrator.create_agent(
    name="coder",
    config=coder_config,
    llm_provider=llm_provider,
    tool_registry=tool_registry,
    input_guardrail=input_guardrail,
    output_guardrail=output_guardrail
)

print("✓ Coder agent created")

In [ ]:
# Test coder agent
if llm_provider:
    response = coder_agent.run(
        "Write a Python function that validates email addresses using regex. "
        "Include docstrings and save it to 'email_validator.py'."
    )
    print("\nCoder Response:")
    print(response)
else:
    print("⚠ No LLM provider available")

## 7. Example 3: Meta-Agent Creating Other Agents

A meta-agent can design and create other specialist agents.

In [ ]:
# Create a meta-agent that can create other agents
meta_agent_prompt = """You are a Meta-Agent that specializes in creating and coordinating other AI agents.

Your capabilities:
1. Analyze tasks and determine what specialist agents are needed
2. Create agent configurations with appropriate prompts and tools
3. Coordinate work between multiple agents
4. Synthesize results from multiple agents

When asked to create an agent, provide:
- Agent name
- Agent role and responsibilities
- Specific system prompt for the agent
- What tools the agent should have access to"""

meta_config = AgentConfig(
    name="Meta-Agent",
    system_prompt=meta_agent_prompt,
    temperature=0.7
)

meta_agent = orchestrator.create_agent(
    name="meta",
    config=meta_config,
    llm_provider=llm_provider,
    input_guardrail=input_guardrail,
    output_guardrail=output_guardrail
)

print("✓ Meta-agent created")

In [ ]:
# Ask meta-agent to design a new specialist agent
if llm_provider:
    response = meta_agent.run(
        "I need to build a system for automated code review. "
        "Design a specialist agent that can review Python code for quality, "
        "security issues, and best practices. Provide the agent configuration."
    )
    print("\nMeta-Agent Response:")
    print(response)
else:
    print("⚠ No LLM provider available")

## 8. Example 4: Prompt Composition and Reuse

Multiple SuperPrompts can be composed together to create more capable agents.

In [ ]:
# Compose multiple SuperPrompts
composed_prompt = prompt_loader.compose_prompts(
    "3tier.md",
    "agentic_ai_framework_coder.md"
)

# Create agent with composed prompt
fullstack_config = AgentConfig(
    name="Full-Stack AI Developer",
    system_prompt=composed_prompt,
    temperature=0.5,
    tools=tool_registry.get_tool_definitions()
)

fullstack_agent = orchestrator.create_agent(
    name="fullstack",
    config=fullstack_config,
    llm_provider=llm_provider,
    tool_registry=tool_registry,
    input_guardrail=input_guardrail,
    output_guardrail=output_guardrail
)

print("✓ Full-stack agent created with composed prompts")
print(f"  Prompt length: {len(composed_prompt)} characters")
print(f"  Composed from: 3tier.md + agentic_ai_framework_coder.md")

## 9. Example 5: Testing Guardrails

Guardrails protect against inappropriate content and sensitive data leakage.

In [ ]:
# Test different guardrail providers
test_inputs = [
    "Write a simple hello world program",  # Safe
    "My API key is sk-1234567890abcdef" * 100,  # Contains sensitive data + too long
    "Create a secure authentication system",  # Safe
]

print("Testing Guardrails:\n")

for i, test_input in enumerate(test_inputs, 1):
    print(f"Test {i}: {test_input[:60]}{'...' if len(test_input) > 60 else ''}")
    
    # Test local guardrail
    result = local_guardrail.check(test_input)
    print(f"  Local: {'✓ PASS' if result.passed else '✗ FAIL'} - {result.message}")
    
    # Test OpenAI guardrail if available
    if openai_guardrail:
        result = openai_guardrail.check(test_input)
        print(f"  OpenAI: {'✓ PASS' if result.passed else '✗ FAIL'} - {result.message}")
    
    print()

## 10. Example 6: Demonstrating Component Swapping

The architecture allows easy swapping of LLM and guardrail providers.

In [ ]:
# Demonstrate component swapping
print("Component Swapping Demo\n")

# Create the same agent with different configurations
simple_prompt = "You are a helpful AI assistant that explains technical concepts clearly."
simple_config = AgentConfig(
    name="Simple Assistant",
    system_prompt=simple_prompt,
    temperature=0.7
)

configs = []

# Configuration 1: OpenAI + OpenAI Guardrails
if openai_provider and openai_guardrail:
    agent_v1 = Agent(
        config=simple_config,
        llm_provider=openai_provider,
        input_guardrail=openai_guardrail,
        output_guardrail=openai_guardrail
    )
    configs.append("Config 1: OpenAI LLM + OpenAI Guardrails")

# Configuration 2: OpenAI + Local Guardrails
if openai_provider:
    agent_v2 = Agent(
        config=simple_config,
        llm_provider=openai_provider,
        input_guardrail=local_guardrail,
        output_guardrail=local_guardrail
    )
    configs.append("Config 2: OpenAI LLM + Local Guardrails")

# Configuration 3: Ollama + Local Guardrails
if ollama_provider:
    agent_v3 = Agent(
        config=simple_config,
        llm_provider=ollama_provider,
        input_guardrail=local_guardrail,
        output_guardrail=local_guardrail
    )
    configs.append("Config 3: Ollama LLM + Local Guardrails")

# Configuration 4: OpenAI + Gemini Guardrails (if available)
if openai_provider and gemini_guardrail:
    agent_v4 = Agent(
        config=simple_config,
        llm_provider=openai_provider,
        input_guardrail=gemini_guardrail,
        output_guardrail=local_guardrail
    )
    configs.append("Config 4: OpenAI LLM + Gemini Input + Local Output Guardrails")

for config in configs:
    print(f"✓ {config}")

print("\n All configurations use the same Agent interface!")
print("This demonstrates the power of abstraction and dependency injection.")

## 11. Example 7: View Files Created by Agents

In [ ]:
# List files created by agents
from pathlib import Path

output_dir = Path("/tmp/agent_outputs")

if output_dir.exists():
    files = list(output_dir.iterdir())
    if files:
        print("Files created by agents:\n")
        for file in files:
            print(f"  📄 {file.name}")
            print(f"     Size: {file.stat().st_size} bytes")
            print(f"     Content preview:")
            with open(file, 'r', encoding='utf-8') as f:
                content = f.read()
                preview = content[:200] + "..." if len(content) > 200 else content
                print(f"     {preview}\n")
    else:
        print("No files created yet. Run the examples above to create some!")
else:
    print("Output directory doesn't exist yet. Run examples with file-saving tools first.")

## Summary

This notebook demonstrates a production-ready implementation of agents with guardrails.

### Key Design Principles

1. **Abstraction**: Abstract base classes for LLM and Guardrail providers
2. **Modularity**: Each component can be swapped independently
3. **Extensibility**: Easy to add new providers, tools, and agents
4. **Reusability**: SuperPrompts can be loaded and composed
5. **Safety**: Input/Output guardrails with multiple provider options

### Swappable Components

- **LLM Providers**: OpenAI, Ollama, or implement your own
- **Guardrail Providers**: OpenAI, Gemini, Local rules, or implement your own
- **Tools**: File operations, custom tools via ToolRegistry
- **Prompts**: Load from SuperPrompt library, compose multiple prompts

### Use Cases Demonstrated

1. ✅ Creating agents from SuperPrompt templates
2. ✅ Multi-agent collaboration
3. ✅ Meta-agents creating other agents
4. ✅ Prompt composition and reuse
5. ✅ File-saving tools
6. ✅ Input/Output guardrails
7. ✅ Component swapping

### Architecture Highlights

**Example: Use Gemini for guardrails on OpenAI prompts**
```python
agent = Agent(
    config=config,
    llm_provider=OpenAIProvider(),      # OpenAI for generation
    input_guardrail=GeminiGuardrailProvider(),  # Gemini for input checks
    output_guardrail=LocalGuardrailProvider()   # Local for output checks
)
```

This demonstrates true separation of concerns and dependency injection!

### Next Steps

- Add more specialized tools (database access, API calls, etc.)
- Implement agent memory/state persistence
- Add more guardrail providers (e.g., Azure Content Safety)
- Create domain-specific agent templates
- Implement agent monitoring and logging
- Add support for streaming responses
- Create a web UI for agent interaction

### References

- [OpenAI Agents SDK Documentation](https://platform.openai.com/docs/guides/agents-sdk)
- [LinkedIn Article on AI Agents with Guardrails](https://www.linkedin.com/posts/liord_ai-agents-are-powerful-but-they-need-activity-7306348810824241152-xSAx)
- Implementation: `my_chat_gpt_utils/agents_sdk.py`